In [1]:
from utils import process_text

In [2]:
import os 
os.chdir('..')
print(os.getcwd())
print(os.listdir())

/home/matcha/new_matcha/Matcha-TTS
['output', '.github', 'scripts', 'synthesis.ipynb', '.ipynb_checkpoints', '.env.example', 'notebook', 'matcha_ljspeech.ckpt', '.project-root', 'checkpoint_epoch=100.ckpt', 'checkpoints', '.git', '.gitignore', 'MANIFEST.in', 'weight_distribution.png', 'requirements.txt', 'LICENSE', '.pre-commit-config.yaml', 'configs', 'logs', 'Loss.ipynb', 'data', 'matcha_tts.egg-info', 'val_synthesis_results', 'notebooks', 'train2.log', 'README.md', 'setup.py', '.pylintrc', 'pyproject.toml', 'Makefile', 'matcha']


In [3]:
import torch
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import soundfile as sf
import sys 

In [8]:
sys.path.append('Matcha_TTS_main/') # add the Matcha_TTS_main (code from the original repo) directory to the path


In [9]:
from model import MatchaTTS 
# for Hifigan
from matcha.hifigan.config import v1
from matcha.hifigan.denoiser import Denoiser
from matcha.hifigan.env import AttrDict
from matcha.hifigan.models import Generator as HiFiGAN

In [10]:
# load the model checkpoints
matcha_checkpoint_path = "logs/train/ljspeech/runs/2026-01-09_12-40-08/checkpoints/last.ckpt"
hifigan_checkpoint_path = "checkpoints/generator_v1"

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def load_model(checkpoint_path):
    model = MatchaTTS.load_from_checkpoint(
        checkpoint_path, 
        map_location=device,
        weights_only=False 
    )
    model.eval()
    return model

model = load_model(matcha_checkpoint_path)
print(f"Model loaded!")

/home/matcha/.local/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


Model loaded!


In [13]:
def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

vocoder = load_vocoder(hifigan_checkpoint_path)
denoiser = Denoiser(vocoder, mode='zeros')

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/tmp/ipykernel_70585/772254795.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case 

Removing weight norm...
